## 講習1 --- IRAFを使ってみる

IRAFのCLコマンドラインを使っての対話的なデータ処理を、 
python + Jupyter notebookからも行うことができます。

pythonからIRAFを使うにはpyrafを利用します。  
pyrafを(便利に)使うためには、  

- ホームディレクトリにirafというディレクトリを作成
- そのディレクトリ内で mkirafを実行し、login.cl を作成
- 必要に応じてそのlogin.clを編集  

をしておきます。 こうしておくと、login.clでの設定がpyraf利用時に反映されます。また、~/iraf/uparm/に各タスク(imexam, imstatなど)のパラメータが保存されます。　　

### モジュールの読み込み

In [1]:
from pyraf import iraf 

これでirafのタスクをpythonで関数として使うことができます。('Warning : sscanf library not ... 'がでることがあります。とりえあず無視してください。)

Unixコマンドを使い、カレントディレクトリおよびその中身を確認することができます。

In [2]:
pwd

'/Users/yas.nakajima/ads2017python'

In [3]:
ls

010604.ffiles/          course1_IRAF_1.ipynb    numpy_basic.ipynb
appendix.ipynb          iraf_apphot.ipynb       pyfits_basic.ipynb
course0_module.ipynb    matplotlib_basic.ipynb


ffilesのディレクトリの中にサンプルの画像FITSファイルがあります。IRAFのコマンド、display, imexam, imstatを使って、それらファイルの表示やデータの吟味、統計量の測定をしてみましょう。

### DS9にFITSデータを表示してみる

別のターミナルからds9を立ち上げておきます。

In [4]:
iraf.display('./010604.ffiles/hf0001.fits', 1)

z1=977.7808 z2=1139.195


In [5]:
iraf.display('./010604.ffiles/hf0002.fits', 2)

z1=971.7075 z2=1131.667


表示されましたね。  
irafモジュールのdisplay()関数を使い、引数としてファイルパス、ds9のフレーム番号、などを指定します。

### imexamも使ってみよう
ds9にFITS画像を表示した状態で、次のコマンドを実行し、マウスカーソルを、星のないところにあてて**m**、あるいは星にあてて**a**をタイプしてみましょう。終わるときには**q**をタイプしてください。

In [6]:
iraf.imexam()

#            SECTION     NPIX     MEAN   MEDIAN   STDDEV      MIN      MAX
   [302:306,619:623]       25    1053.    1052.    14.66    1026.    1085.
   [397:401,638:642]       25    1051.    1052.    12.16    1014.    1075.
   [600:604,341:345]       25    1051.    1052.    14.99    1016.    1076.
   [407:411,343:347]       25    1051.    1052.    19.54    999.2    1086.
   [676:680,596:600]       25    1043.    1050.    19.88    1003.    1079.
#   COL    LINE   COORDINATES
#     R    MAG    FLUX     SKY    PEAK    E   PA BETA ENCLOSED   MOFFAT DIRECT
 586.56  679.89 586.56 679.89
   6.92  13.75  31535.   1052.   4188. 0.08   53 3.18     2.32     2.29   2.31


imexamでは、**r**とか**e**コマンドを使うとさらに別のグラフィックウィンドウが現れます。ブラウザとかの後ろに隠れているかもしれません。

In [7]:
iraf.imexam('./010604.ffiles/hf0001.fits', 1) 

  6.28  12.99  63916.   1058.  10437. 0.08   39 11.2     1.98     2.23   2.09
  5.58  13.16  54479.   1058.  10488. 0.12   33 1.62     1.89     1.81   1.86


### imstat も使ってみる
FITS画像の統計量を調べるタスクimstatを使ってみましょう。

In [8]:
iraf.imstat.unlearn()  #  パラメータをデフォルト値に

In [9]:
iraf.imstat('./010604.ffiles/hf0001.fits')

#               IMAGE      NPIX      MEAN    STDDEV       MIN       MAX
 ./010604.ffiles/hf0001.fits   1048576     893.7     4103.  -199198.    25331.


### eparでパラメータ設定 
**iraf.epar('タスク名')** で、IRAFでおなじみのeparのパラメータ設定の画面が別ウィンドウで開きます。GUIでパラメータ設定ができます。同じGUIのウインドウでHELPも読めます。   
(2017-05 MacOS Sierra + Python3.5.3 + Jupyter Notebook では iraf.epar('display')でUnicodeDecodeErrorが生じる)

In [12]:
iraf.epar('imstat')

### 変数としてパラメータ設定

Cellでタスクの変数にパラメータを代入してやります。この方法だと、使ったパラメータがこのノートブックに残るので、あとになって「この処理でどんなパラメータ使ったっけ？」となったときに助けになります。

In [13]:
iraf.imstat.fields = 'midpt, mean, stddev'
iraf.imstat.lower = 0
iraf.imstat.upper = 10000

In [14]:
iraf.imstat('./010604.ffiles/hf0001.fits')

#     MIDPT      MEAN    STDDEV
      1059.     1060.     56.13


In [16]:
print (iraf.imstat.fields)

midpt, mean, stddev


In [17]:
iraf.imstat.images = './010604.ffiles/hf0001.fits'

In [18]:
iraf.imstat()

#     MIDPT      MEAN    STDDEV
      1059.     1060.     56.13


このノートブック内でhelpを読むこともできます。  
helpの表示が縦に長すぎる場合、左の余白部分(In[]: の下あたり)をクリックするとスクロールバーつきウインドウ表示になります。
(私の環境でブラウザChromeの場合にはデフォルトでスクロールバー表示されます。)

In [20]:
#iraf.help('imstat')   #  Githubでは表示が長くなってしまうのでコメントアウトしておきます

簡単なプログラムで'./010604.ffiles'ディレクトリ内のh-bandのfitsファイル全てについてimstatしましょう。

In [21]:
import glob # pythonの組み込みモジュール。ワイルドカードを使ったファイル処理など。

In [22]:
fitslist = glob.glob('./010604.ffiles/h*fits')

In [23]:
print (fitslist)

['./010604.ffiles/hf0001.fits', './010604.ffiles/hf0002.fits', './010604.ffiles/hf0003.fits', './010604.ffiles/hf0004.fits', './010604.ffiles/hf0005.fits']


In [24]:
for img in fitslist:
    iraf.imstat(img)

#     MIDPT      MEAN    STDDEV
      1059.     1060.     56.13
#     MIDPT      MEAN    STDDEV
      1052.     1053.     52.06
#     MIDPT      MEAN    STDDEV
      1043.     1043.     58.26
#     MIDPT      MEAN    STDDEV
      1018.     1020.     46.33
#     MIDPT      MEAN    STDDEV
      1011.     1012.     42.15


上では、glob.globで抽出したリストをいったん変数に代入しましたが、以下のようにしても十分です。

In [25]:
for img in glob.glob('./010604.ffiles/h*fits'):
    iraf.imstat(img)

#     MIDPT      MEAN    STDDEV
      1059.     1060.     56.13
#     MIDPT      MEAN    STDDEV
      1052.     1053.     52.06
#     MIDPT      MEAN    STDDEV
      1043.     1043.     58.26
#     MIDPT      MEAN    STDDEV
      1018.     1020.     46.33
#     MIDPT      MEAN    STDDEV
      1011.     1012.     42.15


### 練習問題 
未定  